In [3]:
# # LLM-Based Text Summarization Assignment
# **Candidate:** Nitesh Jha
# **Role:** AI Engineering – LLM
# **Company:** Infloso


# STEP 1: Install Libraries
!pip install transformers torch sentencepiece pandas scikit-learn rouge-score nltk

# STEP 2: Imports

import re
import pandas as pd
import numpy as np

from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from rouge_score import rouge_scorer

# STEP 3: FORCE CPU (STABILITY > SPEED)

device = -1
print("Using CPU for stability")

# STEP 4: Load Dataset

with open("/content/drive/MyDrive/reviews.txt", "r", encoding="utf-8") as f:
    reviews = f.readlines()

df = pd.DataFrame(reviews, columns=["review"])
print("Total Reviews:", len(df))

# STEP 5: Clean Text

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_review"] = df["review"].apply(clean_text)

# STEP 6: Sentiment Split (Heuristic)

negative_keywords = [
    "worst", "bad", "error", "issue", "problem",
    "unable", "fake", "waste", "scam", "not able"
]

df["sentiment"] = df["clean_review"].apply(
    lambda x: "negative" if any(k in x for k in negative_keywords) else "positive"
)

positive_reviews = df[df["sentiment"] == "positive"]
negative_reviews = df[df["sentiment"] == "negative"]

print("Positive:", len(positive_reviews), "Negative:", len(negative_reviews))

# STEP 7: Load Summarizer (CPU-SAFE)

summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=device
)

# STEP 8: Safe Chunking (SMALLER CHUNKS)

def chunk_text(text, chunk_size=500):
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield " ".join(words[i:i + chunk_size])

# STEP 9: Summarization Function

def summarize_reviews(reviews, label):
    combined_text = " ".join(reviews.tolist())
    chunks = list(chunk_text(combined_text))
    summaries = []

    for chunk in chunks[:2]:
        summary = summarizer(
            chunk,
            max_length=120,
            min_length=40,
            num_beams=3,
            do_sample=False
        )[0]["summary_text"]
        summaries.append(summary)

    return f"{label} Summary:\n" + " ".join(summaries)

# STEP 10: Generate Balanced Summary

positive_summary = summarize_reviews(positive_reviews["clean_review"], "Positive")
negative_summary = summarize_reviews(negative_reviews["clean_review"], "Negative")

final_summary = positive_summary + "\n\n" + negative_summary

print("\n===== FINAL BALANCED SUMMARY =====\n")
print(final_summary)

# STEP 11: TF-IDF Based Top 5 Reviews

vectorizer = TfidfVectorizer(max_features=1000)
tfidf_matrix = vectorizer.fit_transform(df["clean_review"])

df["importance_score"] = tfidf_matrix.sum(axis=1).A1

top_5_reviews = df.sort_values(
    by="importance_score",
    ascending=False
).head(5)

print("\n===== TOP 5 IMPORTANT REVIEWS =====\n")
for i, review in enumerate(top_5_reviews["review"].tolist(), 1):
    print(f"{i}. {review}\n")

# STEP 12: ROUGE Evaluation

scorer = rouge_scorer.RougeScorer(["rouge1", "rougeL"], use_stemmer=True)

sample_refs = df["clean_review"].sample(5, random_state=42).tolist()
rouge_scores = []

for ref in sample_refs:
    score = scorer.score(ref, final_summary)
    rouge_scores.append(score["rougeL"].fmeasure)

print("Average ROUGE-L:", np.mean(rouge_scores))

# STEP 13: Save Outputs

with open("final_summary.txt", "w", encoding="utf-8") as f:
    f.write(final_summary)

top_5_reviews.to_csv("top_5_reviews.csv", index=False)

print("\nSaved files: final_summary.txt, top_5_reviews.csv")

#  Dataset Exploration

# The dataset consists of 100 user reviews collected from a text file.

# Each line represents one individual user review.

# The reviews contain informal language, mixed sentiment, and real user feedback.

# Basic exploration was done by counting total reviews and inspecting samples.

# This helped understand data quality and structure before processing.

#  Data Preprocessing

# All reviews were converted to lowercase for consistency.

# URLs, special characters, and extra spaces were removed.

# Cleaned text improves model understanding and reduces noise.

# Preprocessing ensures uniform input to the summarization model.

# A cleaned version of each review was stored for further analysis.

#  Summarization Strategy

# Reviews were divided into positive and negative using keyword-based heuristics.

# This avoids bias by capturing both user appreciation and complaints.

# The BART large CNN model was used for abstractive summarization.

# Long text was split into smaller chunks to handle model token limits.

# CPU-only execution was chosen for stability and reproducibility.

#  Results

# A balanced summary was generated combining positive and negative insights.

# The summary highlights key features, benefits, and common issues.

# TF-IDF was used to identify the top 5 most important reviews.

# These reviews represent the most informative user feedback.

# Results were saved as text and CSV files for easy submission.

#  Evaluation

# ROUGE-L score was used to evaluate summary quality.

# It measures overlap between generated summaries and original reviews.

# A sample-based evaluation was performed due to dataset size.

# The score provides a quantitative measure of summary effectiveness.

# This helps validate the summarization approach.

#  Future Work

# Replace heuristic sentiment detection with a trained sentiment classifier.

# Fine-tune the summarization model on domain-specific review data.

# Explore topic-wise or aspect-based summarization.

# Add multilingual review support.

# Improve evaluation using human feedback or additional metrics.


Using CPU for stability
Total Reviews: 100
Positive: 70 Negative: 30


Device set to use cpu



===== FINAL BALANCED SUMMARY =====

Positive Summary:
 infloso is a mind blowing app i have used this app for few months and i am very satisfied with it this app shows only for influencer so can a brand find influencer according to there business i registered through this app but it is not what i was expecting my requests for collaboration are left pending for more than a few months this app is such an amazing one i think every influencer should have this app best app i ever seen in my life and they provide best campaigns to influencer as i m influencer i like there service and customer support you guys must try this app as  infloso is an exceptional paid barter collaboration app that has revolutionized my content creation journey as a content creator. The platform s innovative features and user friendly interface have empowered me to effortlessly connect and collaborate with brands and other creators through infloso. The ability to set deadlines and receive reminders has been a lifes